In [ ]:
# plz don't change

import os

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
!pip install python-docx
import docx
!pip install PyPDF2
import PyPDF2

!pip install transformers datasets
!pip install transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
!pip install sentencepiece

!pip install rouge
from rouge import Rouge
rouge = Rouge()

def calc_rouge_scores(model_summary, reference_summary):
    return rouge.get_scores(model_summary, reference_summary)

# convert the summary in docx into string
def getText(filename):
  doc = docx.Document(filename)
  fullText = []
  for para in doc.paragraphs:
      fullText.append(para.text)
  return ' '.join(fullText)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=fdf0d6e9442cac8a33c9d8229c80ff3fcacee7dc25c3c69d06db61c029aa5538
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# plz don't change
# connecting google drive with this notebook. Choose SMU acc
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
# note: the path may be different for others.
os.chdir('/content/drive/MyDrive/DAP Group 6: Fin Scanners/statements')

MessageError: ignored

In [ ]:
# plz don't change
# shows all the files available in the 'DAP Group 6: Fin Scanners/statements' location. 
# if you don't see it available, pls change the location of your file to the location stated above. 
# make sure the original speech is in .pdf and summary in .docx format (NOT .gdoc)
!ls

In [ ]:
# CHANGE ME PLEASE
# for split_by_paragraph, replace the XXX with the entire paragraph in order.
# for split_by_first_sentence, replace the xxx with the first sentence of each paragraph in order. Join them together under the same inverted commas.
# format: split_by_paragraph = ["XXX", "XXX", "XXX", "XXX"]
# format: split_by_first_sentence = "xxx"
split_by_paragraph = ["Overall economic activity appears to have picked up after edging down in the first quarter. Job gains have been robust in recent months, and the unemployment rate has remained low. Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher energy prices, and broader price pressures.", "The invasion of Ukraine by Russia is causing tremendous human and economic hardship. The invasion and related events are creating additional upward pressure on inflation and are weighing on global economic activity. In addition, COVID-related lockdowns in China are likely to exacerbate supply chain disruptions. The Committee is highly attentive to inflation risks.", "The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. In support of these goals, the Committee decided to raise the target range for the federal funds rate to 1-1/2 to 1-3/4 percent and anticipates that ongoing increases in the target range will be appropriate. In addition, the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage-backed securities, as described in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in May. The Committee is strongly committed to returning inflation to its 2 percent objective.", "In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee would be prepared to adjust the stance of monetary policy as appropriate if risks emerge that could impede the attainment of the Committee’s goals. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments."]
split_by_first_sentence = "Recent indicators point to modest growth in spending and production. Russia’s war against Ukraine is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook."
# if you don't get an error, then you're good to continue
print(split_by_paragraph)
print(split_by_first_sentence)

In [ ]:
# CHANGE ME PLEASE
# for human_summary, replace the XXX with the entire human summary
# format: human_summary = "XXX"
human_summary = "There has been modest growth in spending and production. Job gains have been robust. The pandemic along with Russia’s invasion of Ukraine has caused an elevation of inflation. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent. The Committee will take into account the tightening of monetary policy. It will also monitor further information on public health and labour market conditions. "

In [ ]:
# AFTER RUNNING, COPY THE OUTPUT AND PASTE under the BASELINE column
baseline = calc_rouge_scores(split_by_first_sentence, human_summary)
print(baseline)

In [ ]:
# AFTER RUNNING, COPY THE OUTPUT AND PASTE under the PEGASUS column respectively.
summarizer1 = pipeline(model = "human-centered-summarization/financial-summarization-pegasus")
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

peg_candidate = []
for para in split_by_paragraph:
  input_ids = tokenizer(para, return_tensors="pt").input_ids
  output = model.generate(
      input_ids, 
      min_length=10,
      max_length=3000,
      num_beams=4, 
      early_stopping=True
  )
  peg_candidate.append(tokenizer.decode(output[0], skip_special_tokens=True))
peg_cand = " ".join(peg_candidate)
print(peg_cand)
peg_score = calc_rouge_scores(peg_cand, human_summary)
print(peg_score)

In [ ]:
# AFTER RUNNING, COPY THE OUTPUT AND PASTE under the sshleifer-BART column respectively.
summarizer2 = pipeline(model = "sshleifer/distilbart-cnn-12-6")
sshleifer_candidate = []
for para in split_by_paragraph:
  sshleifer_candidate.append(summarizer2(para, min_length=15,max_length=20)[0]['summary_text'])
sshleifer_cand = " ".join(sshleifer_candidate)
print(sshleifer_cand) 
sshleifer_score = calc_rouge_scores(sshleifer_cand, human_summary)
print(sshleifer_score)

In [ ]:
# AFTER RUNNING, COPY THE OUTPUT AND PASTE under the T5 column respectively.
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
t5_candidate = []
for para in split_by_paragraph: 
  text = "summarize: " + para 
  input_ids = tokenizer.encode(text,return_tensors="pt")
  output_ids = model.generate(
      input_ids, 
      min_length=10,
      max_length=1024
  )
  t5_candidate.append(tokenizer.decode(output_ids[0]))
t5_cand = " ".join(t5_candidate)
t5_cand = t5_cand.replace("<pad>", "")
t5_cand = t5_cand.replace("</s>", "")
print(t5_cand)
t5_score = calc_rouge_scores(t5_cand, human_summary)
print(t5_score)

In [ ]:
# AFTER RUNNING, COPY THE OUTPUT AND PASTE under the GPT2 column respectively.
!pip install bert-extractive-summarizer
from summarizer import Summarizer,TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
gpt2_candidate = []
for para in split_by_paragraph:
  gpt2_candidate.append(GPT2_model(para, min_length=50))
gpt2_cand = ' '.join(gpt2_candidate)
print(gpt2_cand)
gpt2_score = calc_rouge_scores(gpt2_cand, human_summary)
print(gpt2_score)